# Cylindrical Magneto-Coriolis Modes

This notebook builds a complete symbolic-to-numerical pipeline to compute linear **magneto-Coriolis modes** in a rotating cylinder using a **toroidal–poloidal** formulation. Velocity and magnetic perturbations are expanded as Fourier modes $e^{i(k z + m\varphi - \omega t)}$ with radial amplitudes defined by the potentials $(T,P)$ for the flow and $(G,F)$ for the magnetic field. With SageMath’s differential operators in cylindrical coordinates, we construct the linearized momentum and induction equations, and assemble a coupled radial system for $[T,P,G,F]^T$. Boundary conditions at the cylinder radius $s=a$ are imposed by matching the interior field to an exterior current-free field $\boldsymbol B_{\text{ext}}=-\nabla\Phi$. The resulting problem is a generalized eigenvalue problem, which is solved numerically (SciPy) to obtain radial eigenvalues and associated spectra $\omega\,(m, n, k)$.

## 1. Definitions

In [1]:
version()

'SageMath version 10.2, Release Date: 2023-12-03'

In [2]:
%display latex 

In [3]:
E.<s,ph,z> = EuclideanSpace(coordinates='cylindrical')
print(E)
E

Euclidean space E^3


Euclidean space E^3

In [4]:
var('k');
var('m');
var('ω');
var('Omega');
var('t');
var('B0');
var('a');
var('nu');
var('mu');
var('mu_0')
var('rho');
var('eta');
var('C1');

In [5]:
e_s=E.cylindrical_frame()[1]
e_ph=E.cylindrical_frame()[2]
e_z=E.cylindrical_frame()[3]

### Toroidal - Poloidal Fields

In [6]:
T = E.scalar_field(function('T')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='T')
T.display()

T: E^3 → ℝ
   (s, ph, z) ↦ T(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [7]:
G = E.scalar_field(function('G')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='G')
G.display()

G: E^3 → ℝ
   (s, ph, z) ↦ G(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [8]:
P = E.scalar_field(function('P')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='P')
P.display()

P: E^3 → ℝ
   (s, ph, z) ↦ P(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [9]:
F = E.scalar_field(function('F')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='F')
F.display()

F: E^3 → ℝ
   (s, ph, z) ↦ F(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [10]:
from sage.manifolds.operators import *
from sage.manifolds.operators import grad

## 2. Hydrodynamic

### Velocity field

In [11]:
v= curl(T*e_s) + curl(curl(P*e_s))
v.display()

curl(T*e_s)+curl(curl(P*e_s)) = (k^2*s^2*e^(I*m*ph) + m^2*e^(I*m*ph))*P(s)*e^(I*k*z + I*t*ω)/s^2 e_s + (I*k*s^2*T(s)*e^(I*m*ph) + I*m*s*e^(I*m*ph)*d(P)/ds - I*m*P(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s^2 e_ph + (I*k*s*e^(I*m*ph)*d(P)/ds + I*k*P(s)*e^(I*m*ph) - I*m*T(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s e_z

 \begin{align}
    v_s &= \left( k^2 P(s) + \frac{m^2}{s^2}P(s) \right) e^{i(kz + m\varphi + \omega t)}  \\
    v_{\varphi} &= \left(ik T(s) + \frac{im}{s}\frac{\partial P}{\partial s} - \frac{im}{s^2}P(s) \right) e^{i(kz + m\varphi + \omega t)} \\
    v_z &= \left( ik \frac{\partial P}{\partial s} + \frac{ik}{s}P(s) - \frac{im}{s}T(s) \right) e^{i(kz + m\varphi + \omega t)}
\end{align}

In [12]:
dt_v = I*ω*v
dt_v.display()

(I*k^2*s^2*e^(I*m*ph) + I*m^2*e^(I*m*ph))*ω*P(s)*e^(I*k*z + I*t*ω)/s^2 e_s - (k*s^2*T(s)*e^(I*m*ph) + m*s*e^(I*m*ph)*d(P)/ds - m*P(s)*e^(I*m*ph))*ω*e^(I*k*z + I*t*ω)/s^2 e_ph - (k*s*e^(I*m*ph)*d(P)/ds + k*P(s)*e^(I*m*ph) - m*T(s)*e^(I*m*ph))*ω*e^(I*k*z + I*t*ω)/s e_z

### Navier-Stokes equations

In [117]:
NS_vec = dt_v + 2*Omega*e_z.cross(v) - nu*(laplacian(v))
NS_vec.display()

((I*k^2*s^4*e^(I*m*ph) + I*m^2*s^2*e^(I*m*ph))*ω*P(s)*e^(I*k*z) + ((k^4*nu*s^4*e^(I*m*ph) + (2*I*Omega*m + (2*k^2*m^2 + k^2)*nu)*s^2*e^(I*m*ph) + (m^4 - m^2)*nu*e^(I*m*ph))*P(s) - 2*(I*Omega*k*s^4*e^(I*m*ph) + k*m*nu*s^2*e^(I*m*ph))*T(s) + (m^2*nu*s*e^(I*m*ph) - (k^2*nu + 2*I*Omega*m)*s^3*e^(I*m*ph))*d(P)/ds - (k^2*nu*s^4*e^(I*m*ph) + m^2*nu*s^2*e^(I*m*ph))*d^2(P)/ds^2)*e^(I*k*z))*e^(I*t*ω)/s^4 e_s - ((k*s^4*T(s)*e^(I*m*ph) + m*s^3*e^(I*m*ph)*d(P)/ds - m*s^2*P(s)*e^(I*m*ph))*ω*e^(I*k*z) + (I*k*nu*s^4*e^(I*m*ph)*d^2(T)/ds^2 + I*m*nu*s^3*e^(I*m*ph)*d^3(P)/ds^3 + I*k*nu*s^3*e^(I*m*ph)*d(T)/ds - 2*I*m*nu*s^2*e^(I*m*ph)*d^2(P)/ds^2 - (2*Omega*k^2*s^4*e^(I*m*ph) - (3*I*k^2*m*nu - 2*Omega*m^2)*s^2*e^(I*m*ph) + 3*(-I*m^3 + I*m)*nu*e^(I*m*ph))*P(s) + (-I*k^3*nu*s^4*e^(I*m*ph) + (-I*k*m^2 - I*k)*nu*s^2*e^(I*m*ph))*T(s) + (-I*k^2*m*nu*s^3*e^(I*m*ph) + (-I*m^3 + 3*I*m)*nu*s*e^(I*m*ph))*d(P)/ds)*e^(I*k*z))*e^(I*t*ω)/s^4 e_ph - ((k*s^3*e^(I*m*ph)*d(P)/ds + k*s^2*P(s)*e^(I*m*ph) - m*s^2*T(s)*e^(I*m*ph))*ω*e^(I*k*z) + (I*k*nu*s^3*e^(I*m*ph)*d^3(P)/ds^3 + 2*I*k*nu*s^2*e^(I*m*ph)*d^2(P)/ds^2 - I*m*nu*s^2*e^(I*m*ph)*d^2(T)/ds^2 + I*m*nu*s*e^(I*m*ph)*d(T)/ds + (-I*k^3*nu*s^2*e^(I*m*ph) + (-I*k*m^2 + I*k)*nu*e^(I*m*ph))*P(s) + (I*k^2*m*nu*s^2*e^(I*m*ph) + (I*m^3 - I*m)*nu*e^(I*m*ph))*T(s) + (-I*k^3*nu*s^3*e^(I*m*ph) + (-I*k*m^2 - I*k)*nu*s*e^(I*m*ph))*d(P)/ds)*e^(I*k*z))*e^(I*t*ω)/s^3 e_z

In [118]:
NS_1 = e_s.dot(curl(NS_vec))
NS_1.display()

E^3 → ℝ
(s, ph, z) ↦ -((2*I*k*m*s^2*P(s)*e^(I*m*ph) + (-I*k^2*s^4*e^(I*m*ph) - I*m^2*s^2*e^(I*m*ph))*T(s))*ω*e^(I*k*z) - (4*k*m*nu*s^2*e^(I*m*ph)*d^2(P)/ds^2 - 4*k*m*nu*s*e^(I*m*ph)*d(P)/ds - 2*(I*Omega*k^3*s^4*e^(I*m*ph) + (2*k^3*m*nu + I*Omega*k*m^2)*s^2*e^(I*m*ph) + 2*(k*m^3 - k*m)*nu*e^(I*m*ph))*P(s) + (k^4*nu*s^4*e^(I*m*ph) + (2*k^2*m^2 + k^2)*nu*s^2*e^(I*m*ph) + (m^4 - m^2)*nu*e^(I*m*ph))*T(s) - (k^2*nu*s^3*e^(I*m*ph) - m^2*nu*s*e^(I*m*ph))*d(T)/ds - (k^2*nu*s^4*e^(I*m*ph) + m^2*nu*s^2*e^(I*m*ph))*d^2(T)/ds^2)*e^(I*k*z))*e^(I*t*ω)/s^4

In [119]:
NS_2 = e_s.dot(curl(curl(NS_vec)))
NS_2.display()

E^3 → ℝ
(s, ph, z) ↦ ((-2*I*k*m*s^4*T(s)*e^(I*m*ph) + (I*k^4*s^6*e^(I*m*ph) + (2*I*k^2*m^2 + I*k^2)*s^4*e^(I*m*ph) + (I*m^4 - I*m^2)*s^2*e^(I*m*ph))*P(s) + (-I*k^2*s^5*e^(I*m*ph) + I*m^2*s^3*e^(I*m*ph))*d(P)/ds + (-I*k^2*s^6*e^(I*m*ph) - I*m^2*s^4*e^(I*m*ph))*d^2(P)/ds^2)*ω*e^(I*k*z) + (4*k*m*nu*s^4*e^(I*m*ph)*d^2(T)/ds^2 - 4*k*m*nu*s^3*e^(I*m*ph)*d(T)/ds + (k^6*nu*s^6*e^(I*m*ph) + (4*I*Omega*k^2*m + (3*k^4*m^2 + 2*k^4)*nu)*s^4*e^(I*m*ph) + 3*(k^2*m^4 - k^2)*nu*s^2*e^(I*m*ph) + (m^6 - 10*m^4 + 9*m^2)*nu*e^(I*m*ph))*P(s) + 2*(-I*Omega*k^3*s^6*e^(I*m*ph) - (2*k^3*m*nu + I*Omega*k*m^2)*s^4*e^(I*m*ph) - 2*(k*m^3 - k*m)*nu*s^2*e^(I*m*ph))*T(s) - (2*k^4*nu*s^5*e^(I*m*ph) - (4*k^2*m^2 + 3*k^2)*nu*s^3*e^(I*m*ph) - 3*(2*m^4 - 3*m^2)*nu*s*e^(I*m*ph))*d(P)/ds - (2*k^4*nu*s^6*e^(I*m*ph) + (4*k^2*m^2 + 3*k^2)*nu*s^4*e^(I*m*ph) + (2*m^4 - 5*m^2)*nu*s^2*e^(I*m*ph))*d^2(P)/ds^2 + 2*(k^2*nu*s^5*e^(I*m*ph) - m^2*nu*s^3*e^(I*m*ph))*d^3(P)/ds^3 + (k^2*nu*s^6*e^(I*m*ph) + m^2*nu*s^4*e^(I*m*ph))*d^4(P)/ds^4)*e^(I*k*z))*e^(I*t*ω)/s^6

### Eigenvalue system

In [120]:
NS_1_eq = (NS_1.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
NS_1_eq.full_simplify().show()

(4*k*m*nu*s^2*diff(P(s), s, s) - 4*k*m*nu*s*diff(P(s), s) + (-2*I*k*m*s^2*P(s) + (I*k^2*s^4 + I*m^2*s^2)*T(s))*ω + 2*(-I*Omega*k^3*s^4 - (2*k^3*m*nu + I*Omega*k*m^2)*s^2 - 2*(k*m^3 - k*m)*nu)*P(s) + (k^4*nu*s^4 + (2*k^2*m^2 + k^2)*nu*s^2 + (m^4 - m^2)*nu)*T(s) - (k^2*nu*s^3 - m^2*nu*s)*diff(T(s), s) - (k^2*nu*s^4 + m^2*nu*s^2)*diff(T(s), s, s))/s^2 == 0

In [121]:
NS_2_eq = (NS_2.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
NS_2_eq.full_simplify().show()

(4*k*m*nu*s^4*diff(T(s), s, s) - 4*k*m*nu*s^3*diff(T(s), s) + (-2*I*k*m*s^4*T(s) - (-I*k^4*s^6 + (-2*I*k^2*m^2 - I*k^2)*s^4 + (-I*m^4 + I*m^2)*s^2)*P(s) - (I*k^2*s^5 - I*m^2*s^3)*diff(P(s), s) - (I*k^2*s^6 + I*m^2*s^4)*diff(P(s), s, s))*ω + (k^6*nu*s^6 - (-4*I*Omega*k^2*m - (3*k^4*m^2 + 2*k^4)*nu)*s^4 + 3*(k^2*m^4 - k^2)*nu*s^2 + (m^6 - 10*m^4 + 9*m^2)*nu)*P(s) - 2*(I*Omega*k^3*s^6 + (2*k^3*m*nu + I*Omega*k*m^2)*s^4 + 2*(k*m^3 - k*m)*nu*s^2)*T(s) - (2*k^4*nu*s^5 - (4*k^2*m^2 + 3*k^2)*nu*s^3 - 3*(2*m^4 - 3*m^2)*nu*s)*diff(P(s), s) - (2*k^4*nu*s^6 + (4*k^2*m^2 + 3*k^2)*nu*s^4 + (2*m^4 - 5*m^2)*nu*s^2)*diff(P(s), s, s) + 2*(k^2*nu*s^5 - m^2*nu*s^3)*diff(P(s), s, s, s) + (k^2*nu*s^6 + m^2*nu*s^4)*diff(P(s), s, s, s, s))/s^4 == 0

\begin{equation}
A(s) \begin{bmatrix}
T(s) \\
P(s) \end{bmatrix} 
= \omega B(s)
\begin{bmatrix}
T(s) \\
P(s) \end{bmatrix}
\end{equation}


\begin{equation}
\begin{bmatrix}
0 & -2(i \Omega k^3 s^2 + i \Omega k m^2) \\
-2(i \Omega k^{3} s^{2} + i \Omega k m^{2}) & -4i \Omega k^2 m \\
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \end{bmatrix}
= \omega 
\begin{bmatrix}
i(k^2 s^2 + m^2) & 2 i k m \\
2 i k m &
\dfrac{-(-i k^4 s^4 - i m^4 + (-2 i k^2 m^2 - i k^2) s^2 + i m^2) - (i k^2 s^3 - i m^2 s) \dfrac{\partial}{\partial s} - (i k^2 s^4 + i m^2 s^2) \dfrac{\partial ^2}{\partial s^2}}{s^2}
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \end{bmatrix}
\end{equation}


## 3. Greenspan's solution : $B_0 = 0$

We need to find the solutions $\xi$ of the equation:
\begin{equation}
    \xi\frac{d}{d\xi}J_{\lvert m \rvert}(\xi) + m\left( 1 + \frac{\xi^2 h^2}{n_{ax}^2\pi^2 a^2}\right)^{1/2} J_{\lvert m \rvert}(\xi) = 0
\end{equation}
With the corresponding eigenvalues : 
\begin{equation}
    \lambda_{knm} = 2 \left( 1 + \frac{\xi^2_{knm}h^2}{n_{ax}^2\pi^2 a^2}\right)^{-1/2}
\end{equation}

## 4. Magnetohydrodynamic

### Magnetic field

In [15]:
b = curl(G*e_s) + curl(curl(F*e_s))
b.display()

curl(G*e_s)+curl(curl(F*e_s)) = (k^2*s^2*e^(I*m*ph) + m^2*e^(I*m*ph))*F(s)*e^(I*k*z + I*t*ω)/s^2 e_s + (I*k*s^2*G(s)*e^(I*m*ph) + I*m*s*e^(I*m*ph)*d(F)/ds - I*m*F(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s^2 e_ph + (I*k*s*e^(I*m*ph)*d(F)/ds + I*k*F(s)*e^(I*m*ph) - I*m*G(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s e_z

In [16]:
b_s = e_s.dot(b)
b_s.display()

e_s.curl(G*e_s)+curl(curl(F*e_s)): E^3 → ℝ
   (s, ph, z) ↦ (k^2*s^2*e^(I*m*ph) + m^2*e^(I*m*ph))*F(s)*e^(I*k*z + I*t*ω)/s^2

In [17]:
dt_b = I*ω*b

In [18]:
dt_b_s = e_s.dot(dt_b)
dt_b_s.display()

E^3 → ℝ
(s, ph, z) ↦ (I*k^2*s^2*e^(I*m*ph) + I*m^2*e^(I*m*ph))*ω*F(s)*e^(I*k*z + I*t*ω)/s^2

### Equations of motion

#### Magnetic tension

We need to calculate $\nabla \times (ikB_0 \vec{b})\lvert_s$ and $\nabla \times (\nabla \times ikB_0 \vec{b})\lvert_s$ to add to the N-S equations.

In [122]:
MHD_1 = NS_1 - (1/(mu_0*rho))*e_s.dot(curl(I*k*B0*b))

In [123]:
MHD_1_eq = (MHD_1.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
print(latex(MHD_1_eq.full_simplify()))

\frac{4 \, k m \mu_{0} \nu \rho s^{2} \frac{\partial^{2}}{(\partial s)^{2}}P\left(s\right) + 2 i \, B_{0} k^{2} m s^{2} F\left(s\right) - 4 \, k m \mu_{0} \nu \rho s \frac{\partial}{\partial s}P\left(s\right) + {\left(-2 i \, k m \mu_{0} \rho s^{2} P\left(s\right) + {\left(i \, k^{2} \mu_{0} \rho s^{4} + i \, m^{2} \mu_{0} \rho s^{2}\right)} T\left(s\right)\right)} ω + {\left(-i \, B_{0} k^{3} s^{4} - i \, B_{0} k m^{2} s^{2}\right)} G\left(s\right) + 2 \, {\left(-i \, \Omega k^{3} \mu_{0} \rho s^{4} - 2 \, {\left(k m^{3} - k m\right)} \mu_{0} \nu \rho - {\left(2 \, k^{3} m \mu_{0} \nu + i \, \Omega k m^{2} \mu_{0}\right)} \rho s^{2}\right)} P\left(s\right) + {\left(k^{4} \mu_{0} \nu \rho s^{4} + {\left(2 \, k^{2} m^{2} + k^{2}\right)} \mu_{0} \nu \rho s^{2} + {\left(m^{4} - m^{2}\right)} \mu_{0} \nu \rho\right)} T\left(s\right) - {\left(k^{2} \mu_{0} \nu \rho s^{3} - m^{2} \mu_{0} \nu \rho s\right)} \frac{\partial}{\partial s}T\left(s\right) - {\left(k^{2} \mu_{0} \nu \rho s^{4} + m^{

In [124]:
MHD_1_eq.full_simplify().lhs().coefficients(ω)

[[k^4*nu*s^2*T(s) - 2*I*Omega*k^3*s^2*P(s) - 4*k^3*m*nu*P(s) + 2*k^2*m^2*nu*T(s) - k^2*nu*s^2*diff(T(s), s, s) - I*B0*k^3*s^2*G(s)/(mu_0*rho) - 2*I*Omega*k*m^2*P(s) - k^2*nu*s*diff(T(s), s) - 4*k*m^3*nu*P(s)/s^2 + k^2*nu*T(s) + m^4*nu*T(s)/s^2 + 4*k*m*nu*diff(P(s), s, s) - m^2*nu*diff(T(s), s, s) + 2*I*B0*k^2*m*F(s)/(mu_0*rho) - I*B0*k*m^2*G(s)/(mu_0*rho) - 4*k*m*nu*diff(P(s), s)/s + m^2*nu*diff(T(s), s)/s + 4*k*m*nu*P(s)/s^2 - m^2*nu*T(s)/s^2,
  0],
 [I*k^2*s^2*T(s) - 2*I*k*m*P(s) + I*m^2*T(s), 1]]

In [ ]:
# EQ 1 :
A[row, T_idx[j]] = k**4 * mu * sj**2 + 2* k**2 * m**2 * mu - k**2 * mu * sj**2 * D2[j] - k**2 * mu * sj * D[j] + k**2 * mu + 1/s**2 * m**4 * mu - m**2 *mu* D2[j] + 1/s * m**2 * mu * D[j] - 1/s**2 * m**2 * mu
A[row, P_idx[j]] = -2*1j*Omega_rot* k**3 * sj**2 - 4*k**3 * m * mu - 2j*Omega_rot*k* m**2 - 1/s**2 * 4*k* m**3 * mu + 4*k*m*mu* D2[j] - 1/s * 4*k*m*mu* D[j] + 1/s**2 * 4*k*m*mu
A[row, G_idx[j]] = 1/mu0 * 1j*B0 * K**3 * rho * sj**2 - 1j/mu0 * B0 * k*2 * m**2 * rho
A[row, F_idx[j]] = 1/mu0 * 2j * B0 * k**2 * m * rho
# ω
Bm[row, T_idx[j]] = 1j*(k**2 * sj**2 + m**2)
Bm[row, P_idx[j]] = - 2j*k*m
Bm[row, G_idx[j]] = 0
Bm[row, F_idx[j]] =  0

In [125]:
MHD_2 = NS_2 - (1/(mu_0*rho))*e_s.dot(curl(curl(I*k*B0*b)))

In [126]:
MHD_2_eq = (MHD_2.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
print(latex(MHD_2_eq.full_simplify()))

\frac{4 \, k m \mu_{0} \nu \rho s^{4} \frac{\partial^{2}}{(\partial s)^{2}}T\left(s\right) + 2 i \, B_{0} k^{2} m s^{4} G\left(s\right) - 4 \, k m \mu_{0} \nu \rho s^{3} \frac{\partial}{\partial s}T\left(s\right) + {\left(-2 i \, k m \mu_{0} \rho s^{4} T\left(s\right) - {\left(-i \, k^{4} \mu_{0} \rho s^{6} + {\left(-2 i \, k^{2} m^{2} - i \, k^{2}\right)} \mu_{0} \rho s^{4} + {\left(-i \, m^{4} + i \, m^{2}\right)} \mu_{0} \rho s^{2}\right)} P\left(s\right) - {\left(i \, k^{2} \mu_{0} \rho s^{5} - i \, m^{2} \mu_{0} \rho s^{3}\right)} \frac{\partial}{\partial s}P\left(s\right) - {\left(i \, k^{2} \mu_{0} \rho s^{6} + i \, m^{2} \mu_{0} \rho s^{4}\right)} \frac{\partial^{2}}{(\partial s)^{2}}P\left(s\right)\right)} ω - {\left(i \, B_{0} k^{5} s^{6} + {\left(2 i \, B_{0} k^{3} m^{2} + i \, B_{0} k^{3}\right)} s^{4} + {\left(i \, B_{0} k m^{4} - i \, B_{0} k m^{2}\right)} s^{2}\right)} F\left(s\right) + {\left(k^{6} \mu_{0} \nu \rho s^{6} + 3 \, {\left(k^{2} m^{4} - k^{2}\right)} \mu_{0}

In [127]:
MHD_2_eq.full_simplify().lhs().coefficients(ω)

[[k^6*nu*s^2*P(s) + 3*k^4*m^2*nu*P(s) - 2*k^4*nu*s^2*diff(P(s), s, s) - I*B0*k^5*s^2*F(s)/(mu_0*rho) - 2*I*Omega*k^3*s^2*T(s) - 2*k^4*nu*s*diff(P(s), s) + 2*k^4*nu*P(s) + 3*k^2*m^4*nu*P(s)/s^2 - 4*k^3*m*nu*T(s) - 4*k^2*m^2*nu*diff(P(s), s, s) + k^2*nu*s^2*diff(P(s), s, s, s, s) - 2*I*B0*k^3*m^2*F(s)/(mu_0*rho) + 4*I*Omega*k^2*m*P(s) - 2*I*Omega*k*m^2*T(s) + I*B0*k^3*s^2*diff(F(s), s, s)/(mu_0*rho) + 4*k^2*m^2*nu*diff(P(s), s)/s + 2*k^2*nu*s*diff(P(s), s, s, s) + m^6*nu*P(s)/s^4 - 4*k*m^3*nu*T(s)/s^2 + I*B0*k^3*s*diff(F(s), s)/(mu_0*rho) - 3*k^2*nu*diff(P(s), s, s) - 2*m^4*nu*diff(P(s), s, s)/s^2 + m^2*nu*diff(P(s), s, s, s, s) + 4*k*m*nu*diff(T(s), s, s) - I*B0*k^3*F(s)/(mu_0*rho) - I*B0*k*m^4*F(s)/(mu_0*rho*s^2) + 2*I*B0*k^2*m*G(s)/(mu_0*rho) + I*B0*k*m^2*diff(F(s), s, s)/(mu_0*rho) + 6*m^4*nu*diff(P(s), s)/s^3 + 3*k^2*nu*diff(P(s), s)/s - 2*m^2*nu*diff(P(s), s, s, s)/s - 4*k*m*nu*diff(T(s), s)/s - 10*m^4*nu*P(s)/s^4 - 3*k^2*nu*P(s)/s^2 + 4*k*m*nu*T(s)/s^2 - I*B0*k*m^2*diff(F(s), s)/(mu_0*rho*s) + 5*m^2*nu*diff(P(s), s, s)/s^2 + I*B0*k*m^2*F(s)/(mu_0*rho*s^2) - 9*m^2*nu*diff(P(s), s)/s^3 + 9*m^2*nu*P(s)/s^4,
  0],
 [I*k^4*s^2*P(s) + 2*I*k^2*m^2*P(s) - I*k^2*s^2*diff(P(s), s, s) - I*k^2*s*diff(P(s), s) + I*k^2*P(s) + I*m^4*P(s)/s^2 - 2*I*k*m*T(s) - I*m^2*diff(P(s), s, s) + I*m^2*diff(P(s), s)/s - I*m^2*P(s)/s^2,
  1]]

In [ ]:
# EQ 2 :
A[row, T_idx[j]] = - 2j*Omega_rot*k**3 *s**2 - 4*k**3 * m*mu - 2j*Omega_rot*k*m**2 - 1/sj**2 * 4*k*m**3 * mu + 4*k*m*mu*D2[j] - 4/sj *k*m*mu*D[j] + 4/sj**2 *k*m*mu
A[row, P_idx[j]] = k**6 * mu * sj**2 + 3*k**4 * m**2 * mu - 2*k**4 * mu*sj**2 * D2[j] - 2*k**4 * mu *sj * D[j] + 2*k**4 *mu + 1/sj**2 * 3*k**2 * m**4 * mu - 4*k**2 * m**2 * mu * D2[j] + k**2 *mu* sj**2 *D4[j] + 4j*Omega_rot*k**2 *m + 4/sj *k**2 * m**2 * mu * D[j] + 2*k**2 * mu *sj *D3[j] + 1/sj**4 * m**6 * mu - 3*k**2 *mu*D2[j] - 1/sj**2 * 2*m**4 * mu * D2[j] + m**2 * mu * D4[j] + 6/sj**3 * m**4 *mu*D[j] + 1/sj * 3*k**2 *mu*D[j] - 2/sj * m**2 *mu*D3[j] -10/sj**4 *m**4 * mu - 3/sj**2 *k**2 *mu 5/sj**2 * m**2 *mu*D2[j] - 9/sj**3 * m**2 * mu*D[j] + 9/sj**4 * m**2 * mu
A[row, G_idx[j]] = 2j/mu0 * B0*k**2 *m*rho
A[row, F_idx[j]] = - 1j/mu0 * B0* k**5 * rho * s**2 - 2j/mu0 * B0*k**3 * m**2 *rho + 1j/mu0 * B0 *k**3 *rho*sj**2 * D2[j] + 1j/mu0 * B0*k**3 *rho*sj* D[j] - 1j/mu0 * B0*k**3 *rho* - 1j/(mu0*s**2) * B0*k*m**4 * rho + 1j/mu0 * B0*k*m**2 *rho * D2[j] - 1j/(mu0*sj) * B0*k*m**2*rho*D[j] + 1j/(mu0*sj**2) * B0*k*m**2 * rho
# ω 
Bm[row, T_idx[j]] = - 2j*k*m
Bm[row, P_idx[j]] = 1j*k**4 * sj**2 + 2j*k**2 *m**2 - 1j*k**2 * sj**2 * D2[j] - 1j*k**2 * sj*D[j] + 1j*k**2 + 1j/sj**2 * m**4 - 1j*m**2 * D2[j] + 1j/sj * m**2 * D[j] - 1j/sj**2 * m**2
Bm[row, G_idx[j]] = 0
Bm[row, F_idx[j]] = 0

### Induction equation

\begin{equation}
    \frac{\partial}{\partial t}\vec{b}  = \left( \vec{B}_0 \cdot \nabla\right)\vec{v} + \eta \nabla^2 \vec{b}
\end{equation}

In [128]:
Ind_1 = e_s.dot(dt_b - B0*I*k*v - eta*laplacian(b))
Ind_1.display()

E^3 → ℝ
(s, ph, z) ↦ ((I*k^2*s^4*e^(I*m*ph) + I*m^2*s^2*e^(I*m*ph))*ω*F(s)*e^(I*k*z) - (2*eta*k*m*s^2*G(s)*e^(I*m*ph) - (eta*k^4*s^4*e^(I*m*ph) + (2*eta*k^2*m^2 + eta*k^2)*s^2*e^(I*m*ph) + (eta*m^4 - eta*m^2)*e^(I*m*ph))*F(s) - (-I*B0*k^3*s^4*e^(I*m*ph) - I*B0*k*m^2*s^2*e^(I*m*ph))*P(s) + (eta*k^2*s^3*e^(I*m*ph) - eta*m^2*s*e^(I*m*ph))*d(F)/ds + (eta*k^2*s^4*e^(I*m*ph) + eta*m^2*s^2*e^(I*m*ph))*d^2(F)/ds^2)*e^(I*k*z))*e^(I*t*ω)/s^4

In [129]:
Ind_1_eq = (Ind_1.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
print(latex(Ind_1_eq.full_simplify()))

-\frac{2 \, \eta k m s^{2} G\left(s\right) + {\left(-i \, k^{2} s^{4} - i \, m^{2} s^{2}\right)} ω F\left(s\right) - {\left(\eta k^{4} s^{4} + \eta m^{4} - \eta m^{2} + {\left(2 \, \eta k^{2} m^{2} + \eta k^{2}\right)} s^{2}\right)} F\left(s\right) - {\left(-i \, B_{0} k^{3} s^{4} - i \, B_{0} k m^{2} s^{2}\right)} P\left(s\right) + {\left(\eta k^{2} s^{3} - \eta m^{2} s\right)} \frac{\partial}{\partial s}F\left(s\right) + {\left(\eta k^{2} s^{4} + \eta m^{2} s^{2}\right)} \frac{\partial^{2}}{(\partial s)^{2}}F\left(s\right)}{s^{2}} = 0


In [130]:
Ind_1_eq.full_simplify().lhs().coefficients(ω)

[[eta*k^4*s^2*F(s) - I*B0*k^3*s^2*P(s) + 2*eta*k^2*m^2*F(s) - eta*k^2*s^2*diff(F(s), s, s) - I*B0*k*m^2*P(s) - eta*k^2*s*diff(F(s), s) + eta*k^2*F(s) + eta*m^4*F(s)/s^2 - 2*eta*k*m*G(s) - eta*m^2*diff(F(s), s, s) + eta*m^2*diff(F(s), s)/s - eta*m^2*F(s)/s^2,
  0],
 [I*k^2*s^2*F(s) + I*m^2*F(s), 1]]

In [ ]:
# EQ 3 :
A[row, T_idx[j]] = 0
A[row, P_idx[j]] = -1j*B0*k**3 * sj**4 - 1j*B0*k*m**2 * sj**2
A[row, G_idx[j]] = -2*eta*k*m*sj**2
coef0F = (eta*k**4 *sj**4 + eta*m**4 - eta*m**2 + (2*eta*k**2 * m**2 + eta*k**2)*sj**2)
coef1F = -(eta*k**2 * sj**3 -eta*m**2 * sj)
coef2F = -(eta*k**2 * sj**4 + eta*m**2 *sj**2)
A[row, F_idx] += coef0F * np.eye(n_p)[j] + coef1F * D[j] + coef2F * D2[j]
# ω
Bm[row, T_idx[j]] = 0
Bm[row, P_idx[j]] = 0
Bm[row, G_idx[j]] = 0
Bm[row, F_idx[j]] = 1j*k**2 * sj**4 + 1j*m**2 *sj**2

In [131]:
Ind_2 = e_s.dot(curl(dt_b - B0*I*k*v - eta*laplacian(b)))
Ind_2.display()

E^3 → ℝ
(s, ph, z) ↦ -((2*I*k*m*s^2*F(s)*e^(I*m*ph) + (-I*k^2*s^4*e^(I*m*ph) - I*m^2*s^2*e^(I*m*ph))*G(s))*ω*e^(I*k*z) + (-2*I*B0*k^2*m*s^2*P(s)*e^(I*m*ph) - 4*eta*k*m*s^2*e^(I*m*ph)*d^2(F)/ds^2 + 4*eta*k*m*s*e^(I*m*ph)*d(F)/ds + 4*(eta*k^3*m*s^2*e^(I*m*ph) + (eta*k*m^3 - eta*k*m)*e^(I*m*ph))*F(s) - (eta*k^4*s^4*e^(I*m*ph) + (2*eta*k^2*m^2 + eta*k^2)*s^2*e^(I*m*ph) + (eta*m^4 - eta*m^2)*e^(I*m*ph))*G(s) + (I*B0*k^3*s^4*e^(I*m*ph) + I*B0*k*m^2*s^2*e^(I*m*ph))*T(s) + (eta*k^2*s^3*e^(I*m*ph) - eta*m^2*s*e^(I*m*ph))*d(G)/ds + (eta*k^2*s^4*e^(I*m*ph) + eta*m^2*s^2*e^(I*m*ph))*d^2(G)/ds^2)*e^(I*k*z))*e^(I*t*ω)/s^4

In [132]:
Ind_2_eq = (Ind_2.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
print(latex(Ind_2_eq.full_simplify()))

\frac{2 i \, B_{0} k^{2} m s^{2} P\left(s\right) + 4 \, \eta k m s^{2} \frac{\partial^{2}}{(\partial s)^{2}}F\left(s\right) - 4 \, \eta k m s \frac{\partial}{\partial s}F\left(s\right) + {\left(-2 i \, k m s^{2} F\left(s\right) + {\left(i \, k^{2} s^{4} + i \, m^{2} s^{2}\right)} G\left(s\right)\right)} ω - 4 \, {\left(\eta k^{3} m s^{2} + \eta k m^{3} - \eta k m\right)} F\left(s\right) + {\left(\eta k^{4} s^{4} + \eta m^{4} - \eta m^{2} + {\left(2 \, \eta k^{2} m^{2} + \eta k^{2}\right)} s^{2}\right)} G\left(s\right) + {\left(-i \, B_{0} k^{3} s^{4} - i \, B_{0} k m^{2} s^{2}\right)} T\left(s\right) - {\left(\eta k^{2} s^{3} - \eta m^{2} s\right)} \frac{\partial}{\partial s}G\left(s\right) - {\left(\eta k^{2} s^{4} + \eta m^{2} s^{2}\right)} \frac{\partial^{2}}{(\partial s)^{2}}G\left(s\right)}{s^{2}} = 0


In [133]:
Ind_2_eq.full_simplify().lhs().coefficients(ω)

[[eta*k^4*s^2*G(s) - I*B0*k^3*s^2*T(s) - 4*eta*k^3*m*F(s) + 2*eta*k^2*m^2*G(s) - eta*k^2*s^2*diff(G(s), s, s) + 2*I*B0*k^2*m*P(s) - I*B0*k*m^2*T(s) - eta*k^2*s*diff(G(s), s) - 4*eta*k*m^3*F(s)/s^2 + eta*k^2*G(s) + eta*m^4*G(s)/s^2 + 4*eta*k*m*diff(F(s), s, s) - eta*m^2*diff(G(s), s, s) - 4*eta*k*m*diff(F(s), s)/s + eta*m^2*diff(G(s), s)/s + 4*eta*k*m*F(s)/s^2 - eta*m^2*G(s)/s^2,
  0],
 [I*k^2*s^2*G(s) - 2*I*k*m*F(s) + I*m^2*G(s), 1]]

In [69]:
# EQ 4 :
A[row, T_idx[j]] = -1j*B0*k**3 *sj**4 - 1j*k*B0*k*m**2 * sj**2
A[row, P_idx[j]] = 2j*B0*k**2 *m * sj**2
coef0_G = (eta*k**4 *sj**4 + eta*m**4 - eta*m**2 + (2*eta*k**2 * m**2 + eta*k**2)*sj**2)
coef1_G = -(eta*k**2 * s**3 - eta*m**2 *sj)
coef2_G = -(eta*k**2 * sj**4 + eta*m**2 * sj**2)
A[row, G_idx] += coef0_G * np.eye(n_p)[j] + coef1_G * D[j] + coef2_G * D2[j]
coef0_F = -4*(eta*k**3 * m * sj**2 + eta*k*m**3 - eta*k*m)
coef1_F = -4*eta*k*m*sj
coef2_F = 4*eta*k*m* sj**2
A[row, F_idx] += coef0_F * np.eye(n_p)[j] + coef1_F * D[j] + coef2_F * D2[j]
# ω
B[row, T_idx[j]] = 0
B[row, P_idx[j]] = 0
B[row, G_idx[j]] = 1j*k**2 *sj**4 + 1j*m**2 *sj**2
B[row, F_idx[j]] = -2j*k*m*sj**2

NameError: name 'sj' is not defined

### Eigenvalue system

\begin{equation}
\mathcal{A(s)} \begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix} 
= \omega \mathcal{B(s)}
\begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix}
\end{equation}


\begin{equation}
\begin{bmatrix}
0 & -2 i \Omega k (k^2 s^2 + m^2) & - i B_0 \rho k (k^2 s^2 + m^2) & + 2 i B_0 k^2 m \rho \\
0 & 4 i \Omega k^2 m & \frac{1}{\mu s^2} 2 i B_0 k^2 m \rho s^2 & \frac{-1}{\mu s^2} \left(i B_0 k^5 \rho s^4 + (2 i B_0 k^3 m^2 + i B_0 k^3) \rho s^2 + (i B_0 k m^4 - i B_0 k m^2) \rho\right) + \frac{-1}{\mu s^2} \left(-i B_0 k^3 \rho s^3 + i B_0 k m^2 \rho s\right) \frac{d}{ds} + \frac{-1}{\mu s^2} \left(-i B_0 k^3 \rho s^4 - i B_0 k m^2 \rho s^2\right) \frac{d^2}{ds^2} \\
0 & -i B_0 k (k^2 s^2 + m^2) & 0 & 0 \\
-i B_0 k (k^2 s^2 + m^2) & 2 i B_0 k^2 m & 0 & 0
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix}
= \omega 
\begin{bmatrix}
i (k^2 s^2 + m^2) & -2 i k m & 0 & 0\\
-2 i k m & i \left(k^4 s^2 + (2k^2 m^2 + k^2) + m^4 - m^2\right) -(i k^2 s + i m^2 1/s) \frac{d}{ds} - (i k^2 s^2 + i m^2) \frac{d^2}{ds^2} & 0 & 0 \\
0 & 0 & 0 & i (k^2 s^2 + m^2) \\
0 & 0 & i (k^2 s^2 + m^2) & -2 i k m
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix}
\end{equation}

### Boundary conditions :

A magnetic flux tube is surrounded by an external magnetic field $\mathbf{B}_{ext}$, which can be assumed to be derived from a potential of the same form as toroidal-poloidal fields. That is, we have 
\begin{equation}
    \mathbf{B}_{ext} = - \nabla \Phi \,\ \text{where} \,\ \Phi = \tilde{\Phi}(s) e^{i(m\varphi + kz + \omega t)}
\end{equation}
Where $\tilde{\Phi}$ is again the Fourier amplitude. Since $\nabla \cdot \mathbf{B} = 0$, we can solve
\begin{equation}
    \nabla^2 \Phi = 0
\end{equation}
such that 
\begin{equation}
    \Phi = C_1 Y_m (-iks) e^{i(m \varphi + kz + \omega t)}
\end{equation}

In [24]:
bs = e_s.dot(b)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
bs.expr().full_simplify().show()

(k^2*s^2 + m^2)*F(s)

In [50]:
bph = e_ph.dot(b)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
bph.expr().full_simplify().show()

I*k*s^2*G(s) + I*m*s*diff(F(s), s) - I*m*F(s)

In [26]:
bz = e_z.dot(b)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
bz.expr().full_simplify().show()

I*k*s^2*diff(F(s), s) + I*k*s*F(s) - I*m*s*G(s)

In [1]:
F_1, dF_1, G_1, k, m, C = var('F_1 dF_1 G_1 k m C')

In [2]:
Phi = function('Phi')(s)
Phi == C1*bessel_Y(m,-I*k*s)

Phi(s) == C1*bessel_Y(m, -I*k*s)

In [73]:
eq_s = bs + C1*Phi.diff(s)
eq1 = (eq_s.expr() == 0)
eq1.show()

(k^2*s^2 + m^2)*F(s) + C1*diff(Phi(s), s) == 0

In [17]:
F_1 == -(C/(k**2 + m**2))*diff(bessel_Y(m,-(I*k)*s),s)

F_1 == -1/2*I*C*k*(bessel_Y(m + 1, -I*k*s) - bessel_Y(m - 1, -I*k*s))/(k^2 + m^2)

In [18]:
dF_1 == -(C/(k**2 + m**2))*diff(bessel_Y(m,-(I*k)*s),s) - C*(bessel_Y(m,-(I*k)*s))

dF_1 == -1/2*I*C*k*(bessel_Y(m + 1, -I*k*s) - bessel_Y(m - 1, -I*k*s))/(k^2 + m^2) - C*bessel_Y(m, -I*k*s)

In [74]:
eq_ph = bph + I*m*C1*Phi.diff(s)
eq2 = (eq_ph.expr() == 0)
eq2.show()

I*k*s^2*G(s) + I*m*s*diff(F(s), s) + I*C1*m*diff(Phi(s), s) - I*m*F(s) == 0

In [75]:
eq_z = bz + I*m*C1*Phi.diff(s)
eq3 = (eq_z.expr() == 0)
eq3.show()

I*k*s^2*diff(F(s), s) + I*k*s*F(s) - I*m*s*G(s) + I*C1*m*diff(Phi(s), s) == 0